In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
hitters=pd.read_csv('/kaggle/input/hof-data/hitters2.csv').drop(columns='Unnamed: 0')
hitters

In [ ]:
names=pd.read_csv('/kaggle/input/the-history-of-baseball/player.csv')
names['name']=names['name_first']+' '+names['name_last']
names=names[['player_id','name']]
names=names[names['player_id'].isin(hitters['player_id'].tolist())]
hitters=hitters.join(names.set_index(['player_id']), on='player_id')
hitters

Just realized that there is no average column...let's add this in there.

In [ ]:
hitters['avg']=hitters['h']/hitters['ab']
hitters

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
plt.figure(figsize=(20,10))
sns.heatmap(hitters.corr(),annot=True,linewidth=0.5)

Now let's make some important changes. This could be considered data normalization or filtering the data as we want to get rid of outliers. There are outside factors that we need to adjust for so that these outliers do no throw of our model. Pete Rose is the all time leader in hits by far, which on paper makes him a lock for the hall of fame, but the scandals of him gambling on games he coached/played in are the reason he's not in. A number of other players are not in clearly due to cheating scandals (Barry Bonds, Sammy Sosa, etc.). 

In [ ]:
p1=sns.lmplot(x='hr', y="percent", data=hitters,hue='inducted',fit_reg=False,height=4,aspect=4)
ax = p1.axes[0,0]
for i in range(len(hitters)):
    ax.text(hitters['hr'][i], hitters['percent'][i], hitters['name'][i],
           fontsize='small',rotation=45)

Can see some pretty clear outliers.

In [ ]:
p1=sns.lmplot(x='h', y="percent", data=hitters,hue='inducted',fit_reg=False,height=4,aspect=4)
ax = p1.axes[0,0]
for i in range(len(hitters)):
    ax.text(hitters['h'][i], hitters['percent'][i], hitters['name'][i],
           fontsize='small',rotation=45)

Pete Rose clearly an outlier.

Barry Bonds (bondsba01), Mark McGwire (mcgwima01), Gary Sheffield (sheffga01), Rafael Palmeiro (palmera01), Sammy Sosa(sosasa01), Pete Rose (rosepe01)

In [ ]:
outliers=['bondsba01','mcgwima01','sheffga01','palmera01','sosasa01','rosepe01']

In [ ]:
hitters_filtered=hitters[-hitters['player_id'].isin(outliers)].reset_index().drop(columns='index')
hitters_filtered

Now the data looks much cleaner

In [ ]:
plt.figure(figsize=(20,10))
sns.heatmap(hitters_filtered.corr(),annot=True,linewidth=0.5)

In [ ]:
df=pd.DataFrame(hitters_filtered.corr()['percent']).reset_index()
df['Beat Threshold']=abs(df['percent'])>0.45

In [ ]:
sns.lmplot(x='index', y="percent", data=df,hue='Beat Threshold',fit_reg=False,height=4,
           aspect=4).set_xticklabels(rotation=90)

Factors are g, ab, r, h, hr, rbi, and bb.

In [ ]:
def scatter(attribute):
    p1=sns.lmplot(x=attribute, y="percent", data=hitters,hue='inducted',fit_reg=False,height=4,aspect=4)
    ax = p1.axes[0,0]
    for i in range(len(hitters)):
        ax.text(hitters[attribute][i], hitters['percent'][i], hitters['name'][i],
               fontsize='small',rotation=45)

In [ ]:
scatter('g')

In [ ]:
scatter('ab')

In [ ]:
scatter('r')

In [ ]:
scatter('h')

In [ ]:
scatter('hr')

In [ ]:
scatter('rbi')

In [ ]:
scatter('bb')

In [ ]:
hitters_filtered.to_csv('hitters_filtered')